##Giorgio Mendoza

###RBE595-S24-S01

##Bayes Filter Project


For this project, the Bayes Filter is used to estimate the probability that a door is open based on sensor readings and actions taken over time. It begins with a prior belief about the state of the door and updates this belief as it receives new information.

When an action is taken, such as "pushing" the door, the prediction step adjusts the belief based on the certainty that the action would open the door. Following an action, the update step incorporates sensor measurements using a probabilistic sensor model, which accounts for the sensor's accuracy in detecting whether the door is open or closed.

The filter iteratively refines the belief state through these steps, leading to an increasingly accurate estimate of whether the door is open. In scenarios where the sensor data or the actions taken are consistent over time, the filter can quickly converge to a high confidence level about the door's state, even with an initially uncertain or erroneous belief.








##Task 1:

Analyze the sensor (measurement) model described in Equation 1. What does this tell us about the
usefulness or reliability of this sensor? Using the style of notation demonstrated in the lecture notes and
Equation 1, write out the mathematical representation of the propagation model: 𝑝(𝑥𝑡|𝑢𝑡, 𝑥𝑡−1) for all
possible prior states and inputs. What does this model indicate about the system dynamics?


For Task 1, we need to analyze the sensor model based on the probabilities given. The sensor model probabilities can be summarized as follows:

The probability of the sensor reading 'open' when the door is actually open (True Positive) is p(Zt = 1 | Xt= 1) = 0.6

The probability of the sensor reading 'closed' when the door is actually open (False Negative) is p(Zt = 0 | Xt= 1) = 0.4

The probability of the sensor reading 'open' when the door is actually closed (False Positive) is p(Zt = 1 | Xt= 0) = 0.2

The probability of the sensor reading 'closed' when the door is actually closed (True Negative) is p(Zt = 0 | Xt= 0) = 0.8

This tells us that the sensor is more reliable at detecting when the door is closed than when it is open, as indicated by a higher true negative rate (0.8) compared to the true positive rate (0.6).



In [3]:
import pandas as pd
import numpy as np
# We will implement a simple function to simulate sensor model
# and calculate posterior probability given a prior and a measurement.

def sensor_model(prior, measurement):
    """
    Calculate posterior probability of door being open based on prior
    and measurement using sensor model.

    Parameters:
    - prior: float, prior probability of door being open
    - measurement: int, sensor measurement (0 for 'closed', 1 for 'open')

    Returns:
    - posterior: float, updated probability of door being open
    """
    # Sensor model probabilities
    p_z_given_x = np.array([[0.8, 0.4],  # p(Z=0|X=0), p(Z=0|X=1)
                            [0.2, 0.6]]) # p(Z=1|X=0), p(Z=1|X=1)

    # Calculate posterior probability using Bayes' Rule
    numerator = p_z_given_x[measurement, 1] * prior
    denominator = (p_z_given_x[measurement, 1] * prior +
                   p_z_given_x[measurement, 0] * (1 - prior))

    posterior = numerator / denominator if denominator != 0 else 0

    return posterior

# Example scenarios to show how sensor model updates belief based on measurements
prior_belief = 0.5  # Let's start with a 50% belief that door is open
measurements = [1, 0, 1, 1, 0]  # Sequence of sensor measurements: open, closed, open, open, closed

# Run through measurements and update belief
updated_beliefs = []
for measurement in measurements:
    posterior = sensor_model(prior_belief, measurement)
    updated_beliefs.append(posterior)
    prior_belief = posterior  # posterior becomes new prior for next measurement

updated_beliefs


[0.7499999999999999,
 0.5999999999999999,
 0.8181818181818181,
 0.9310344827586207,
 0.8709677419354838]

##Task 2:
Implement the Bayes Filter algorithm. This implementation should take as input a sequence of measurements and a
sequence of inputs, and optionally an initial probability that the door is open. Separate sub-methods: one to implement the prediction step (line 3) and one to
implement the measurement update step (line 4). The necessity for breaking it up in this manner will
become more apparent later in the course. Ultimately, your implementation should return the
probability that the door is open (i.e. 𝑏𝑒𝑙(𝑥𝑡 = 1)).


Code must answer the following questions:
1. If the robot always takes the action “do nothing” and always receives the measurement “door
open” how many iterations will it take before the robot is at least 99.99% certain the door is
open?
2. If the robot always takes the action “push” and always receives the measurement “door open”
how many iterations will it take before the robot is at least 99.99% certain the door is open?
3. If the robot always takes the action “push” and always receives the measurement “door closed”
what is the steady state belief about the door? Include both the state and the certainty.

In [17]:
import numpy as np

# Define Bayes filter class
class BayesFilter:
    def __init__(self, prior):
        """
        Initialize Bayes Filter with a prior belief.

        Parameters:
        - prior: float, initial belief that door is open
        """
        self.belief = prior  # belief that door is open

    def prediction_step(self, u_t):
        """
        Perform prediction step of Bayes filter.

        Parameters:
        - u_t: int, action taken at time t (0 for 'do nothing', 1 for 'push')
        """
        # If we push door (u_t = 1), we are certain it remains open
        if u_t == 1:
            self.belief = 1.0

    def update_step(self, z_t):
        # Sensor model probabilities
        p_z_given_x = np.array([[0.8, 0.4],  # p(Z=0|X=0), p(Z=0|X=1)
                                [0.2, 0.6]]) # p(Z=1|X=0), p(Z=1|X=1)

        # Calculate the unnormalized new belief
        numerator = p_z_given_x[z_t, 1] * self.belief
        denominator = (p_z_given_x[z_t, 1] * self.belief + p_z_given_x[z_t, 0] * (1 - self.belief))

        # Normalize the belief
        self.belief = numerator / denominator if denominator != 0 else 0


    def get_belief(self):
        """
        Return current belief state.
        """
        return self.belief


# Now, we'll write a function that uses this BayesFilter to answer questions
def run_filter_scenario(prior, actions, measurements, threshold=0.9999):
    """
    Run Bayes filter for a given set of actions and measurements until threshold is reached.

    Parameters:
    - prior: float, initial belief that door is open
    - actions: list of int, sequence of actions taken over time
    - measurements: list of int, sequence of sensor measurements over time
    - threshold: float, certainty threshold

    Returns:
    - iterations: int, number of iterations needed to reach threshold
    """
    bayes_filter = BayesFilter(prior)
    iterations = 0
    while bayes_filter.get_belief() < threshold:
        bayes_filter.prediction_step(actions[iterations])
        bayes_filter.update_step(measurements[iterations])
        iterations += 1
    return iterations

# Test function with a given prior, actions, and measurements
prior = 0.5  # Starting with a 50% belief that door is open
actions = [0] * 1000  # Placeholder: 'do nothing' action
measurements = [1] * 1000  # Placeholder: 'door open' measurement

In [18]:
def run_filter_scenario(prior, actions, measurements, threshold=0.9999):
    bayes_filter = BayesFilter(prior)
    iterations = 0
    while bayes_filter.get_belief() < threshold:
        print(f"Iteration {iterations}:")
        print(f"  Prior belief: {bayes_filter.get_belief()}")
        bayes_filter.prediction_step(actions[iterations])
        print(f"  After prediction step: {bayes_filter.get_belief()}")
        bayes_filter.update_step(measurements[iterations])
        print(f"  After update step: {bayes_filter.get_belief()}")
        iterations += 1
    return iterations

iterations_q1 = run_filter_scenario(0.5, [0]*10000, [1]*10000, 0.9999)
iterations_q2 = run_filter_scenario(0.5, [1]*10000, [1]*10000, 0.9999)

print("Iterations for Q1:", iterations_q1)
print("Iterations for Q2:", iterations_q2)

Iteration 0:
  Prior belief: 0.5
  After prediction step: 0.5
  After update step: 0.7499999999999999
Iteration 1:
  Prior belief: 0.7499999999999999
  After prediction step: 0.7499999999999999
  After update step: 0.8999999999999999
Iteration 2:
  Prior belief: 0.8999999999999999
  After prediction step: 0.8999999999999999
  After update step: 0.9642857142857143
Iteration 3:
  Prior belief: 0.9642857142857143
  After prediction step: 0.9642857142857143
  After update step: 0.9878048780487805
Iteration 4:
  Prior belief: 0.9878048780487805
  After prediction step: 0.9878048780487805
  After update step: 0.9959016393442623
Iteration 5:
  Prior belief: 0.9959016393442623
  After prediction step: 0.9959016393442623
  After update step: 0.9986301369863014
Iteration 6:
  Prior belief: 0.9986301369863014
  After prediction step: 0.9986301369863014
  After update step: 0.9995429616087751
Iteration 7:
  Prior belief: 0.9995429616087751
  After prediction step: 0.9995429616087751
  After update

##Question 1:

When the robot always takes the action “do nothing” and always receives the measurement “door open,” the Bayes Filter started with a prior belief of 50% that the door is open. With each iteration, the sensor reports that the door is open, gradually increasing this belief. Since the action is "do nothing," the belief change is solely based on the sensor's measurement. It took 9 iterations for the belief to surpass 99.99% certainty. This progression demonstrates the sensor's influence on updating the belief state, with the true positive rate (60% chance of correctly detecting an open door) playing a key role in how quickly the belief converges to near certainty.

##Question 2:

For this scenario, where the robot always takes the action "push" and always receives the measurement "door open," the filter's behavior was notably different. Due to the design of the filter, where a "push" action is assumed to open the door with certainty, the belief was immediately set to 100% certainty after the first action. This resulted in achieving the 99.99% certainty threshold in just 1 iteration. The outcome here highlights the impact of the action on the belief state – the "push" action is considered definitive, overriding the prior belief and any sensor input to set the belief at full certainty.

In [24]:
# Define a modified BayesFilter class that handles both action and sensor uncertainty
class BayesFilter:
    def __init__(self, prior):
        self.belief = prior  # belief that door is open

    def prediction_step(self, u_t):
        # If action is 'push', update belief based on probability of successfully opening door
        if u_t == 1:
            # door will be open with 80% probability if it was closed, stay open if it was already open
            # Plus 20% probability that push fails to open door
            self.belief = 0.8 + 0.2 * self.belief


    def update_step(self, z_t):
        # Sensor model probabilities
        p_z_given_x = np.array([[0.8, 0.2],  # p(Z=0|X=0), p(Z=0|X=1)
                                [0.2, 0.6]]) # p(Z=1|X=0), p(Z=1|X=1)

        # If sensor says door is closed, factor this in strongly, especially if belief was high
        if z_t == 0:
            self.belief *= p_z_given_x[z_t, 0]


    def get_belief(self):
        return self.belief

# Function to find steady state belief
def find_steady_state_belief(prior, action, measurement, delta_threshold=0.0001):
    bayes_filter = BayesFilter(prior)
    last_belief = prior
    iterations = 0
    while True:
        bayes_filter.prediction_step(action)
        bayes_filter.update_step(measurement)
        current_belief = bayes_filter.get_belief()
        # Check if change in belief is smaller than threshold, indicating steady state
        if abs(current_belief - last_belief) < delta_threshold:
            return current_belief, iterations
        last_belief = current_belief
        iterations += 1
        # Safety break to avoid infinite loops in case of unexpected behavior
        if iterations > 10000:
            break
    return last_belief, iterations  # Return belief in case steady state not found

steady_state_belief, iterations_to_steady_state = find_steady_state_belief(0.5, 1, 0)
print("Steady State Belief:", steady_state_belief)
print("Iterations to Steady State:", iterations_to_steady_state)


Steady State Belief: 0.7619003678720001
Iterations to Steady State: 5


##Explanation:

The code defines and utilizes a Bayesian Filter to estimate the state of a door—open or closed—based on sensor measurements and actions taken to alter the state (i.e., pushing the door). The BayesFilter class maintains a belief about the door's state, which it updates in two steps:

Prediction Step: The filter anticipates the outcome of an action without yet considering new sensor data. In the case of a "push" action, it increases the probability that the door is open, reflecting the action's effectiveness. It's not entirely certain because there's a 20% chance that pushing will not open the door.

Update Step: Upon receiving a new sensor measurement, the filter adjusts its belief. This is where the filter accounts for the sensor's reliability, using probabilities that express the sensor's ability to correctly or incorrectly report the door's state.

For Question 3, the filter processes a sequence of "push" actions, each followed by a sensor reading indicating the door is closed. The find_steady_state_belief function iteratively applies these steps until the change in belief between iterations falls below a small threshold, signaling that a steady state has been reached.

The output indicates that after 5 iterations of pushing the door and getting a "closed" reading from the sensor, the filter converges to a steady state belief of approximately 76.19%. This means that even though the door is pushed—which has an 80% chance of opening the door—the consistent "closed" sensor readings prevent the belief from reaching full certainty. The filter has essentially reached a compromise between the expected effect of the action and the contradictory sensor data, reflecting the inherent uncertainties in both.






